# Scenario 1: A single data scientist participating in an ML competition

MLflow setup:

Tracking server: no  
Backend store: local filesystem  
Artifacts store: local filesystem  
The experiments can be explored locally by launching the MLflow UI.  

In [1]:
import mlflow

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///workspaces/codespaces-blank/running-mlflow-examples/mlruns'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///workspaces/codespaces-blank/running-mlflow-examples/mlruns/0', creation_time=1685337969220, experiment_id='0', last_update_time=1685337969220, lifecycle_stage='active', name='Default', tags={}>]

### Creating an experiment and logging a new run

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2023/05/29 05:30:01 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'file:///workspaces/codespaces-blank/running-mlflow-examples/mlruns/919311050741103302/58b1d15cd62747c1b1f8b507b0d60fd6/artifacts'


/home/codespace/.local/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///workspaces/codespaces-blank/running-mlflow-examples/mlruns/919311050741103302', creation_time=1685338201351, experiment_id='919311050741103302', last_update_time=1685338201351, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///workspaces/codespaces-blank/running-mlflow-examples/mlruns/0', creation_time=1685337969220, experiment_id='0', last_update_time=1685337969220, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:

from mlflow.tracking import MlflowClient


client = MlflowClient()

In [8]:
from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")

In [10]:
client.search_registered_models()


[]

In [11]:
client.create_registered_model()

'file:///workspaces/codespaces-blank/running-mlflow-examples/mlruns'

In [13]:
import subprocess
subprocess.Popen(["mlflow", "ui"])

<Popen: returncode: None args: ['mlflow', 'ui']>

[2023-05-29 05:36:55 +0000] [30908] [INFO] Starting gunicorn 20.1.0
[2023-05-29 05:36:55 +0000] [30908] [INFO] Listening at: http://127.0.0.1:5000 (30908)
[2023-05-29 05:36:55 +0000] [30908] [INFO] Using worker: sync
[2023-05-29 05:36:55 +0000] [30909] [INFO] Booting worker with pid: 30909
[2023-05-29 05:36:55 +0000] [30910] [INFO] Booting worker with pid: 30910
[2023-05-29 05:36:55 +0000] [30911] [INFO] Booting worker with pid: 30911
[2023-05-29 05:36:55 +0000] [30912] [INFO] Booting worker with pid: 30912


In [14]:
run_id = '58b1d15cd62747c1b1f8b507b0d60fd6'
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name='iris-model')

Successfully registered model 'iris-model'.
2023/05/29 05:38:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-model, version 1
Created version '1' of model 'iris-model'.


<ModelVersion: aliases=[], creation_timestamp=1685338687415, current_stage='None', description=None, last_updated_timestamp=1685338687415, name='iris-model', run_id='58b1d15cd62747c1b1f8b507b0d60fd6', run_link=None, source='file:///workspaces/codespaces-blank/running-mlflow-examples/mlruns/919311050741103302/58b1d15cd62747c1b1f8b507b0d60fd6/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

2023/05/29 05:40:48 ERROR mlflow.server: Exception on /model-versions/get-artifact [GET]
Traceback (most recent call last):
  File "/opt/conda/envs/mlops/lib/python3.9/site-packages/flask/app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/envs/mlops/lib/python3.9/site-packages/flask/app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/envs/mlops/lib/python3.9/site-packages/flask/app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/envs/mlops/lib/python3.9/site-packages/flask/app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/opt/conda/envs/mlops/lib/python3.9/site-packages/mlflow/server/__init__.py", line 70, in serve_model_version_artifact
    return get_model_version_artifact_handler()
  File "/opt/conda/envs/mlops/lib/python3.9/site-packages/mlflow/server/handlers.py", line